In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore")

# 한글 폰트 설정
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

# 스타일 설정
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
# 데이터 디렉토리 구조 확인
data_dir = "../data"


def explore_data_structure(directory):
    """데이터 디렉토리 구조를 탐색하고 파일 정보를 반환"""
    data_info = {}

    for root, dirs, files in os.walk(directory):
        if files:  # CSV 파일이 있는 경우만
            csv_files = [f for f in files if f.endswith(".csv")]
            if csv_files:
                rel_path = os.path.relpath(root, directory)
                data_info[rel_path] = csv_files

    return data_info


# 데이터 구조 탐색
data_structure = explore_data_structure(data_dir)

print("📊 수집된 데이터 구조:")
print("=" * 50)
for category, files in data_structure.items():
    print(f"\n📁 {category}:")
    for file in files[:5]:  # 처음 5개 파일만 표시
        print(f"  - {file}")
    if len(files) > 5:
        print(f"  ... 외 {len(files)-5}개 파일")
    print(f"  총 {len(files)}개 파일")

In [ ]:
# 주요 데이터셋 로드 함수
def load_dataset(category, feature_name, code=None):
    """특정 카테고리의 데이터셋을 로드"""
    path = os.path.join(data_dir, category, feature_name)

    if code:
        file_path = os.path.join(path, f"{code}.csv")
        if os.path.exists(file_path):
            return pd.read_csv(file_path)
    else:
        # 첫 번째 CSV 파일 로드
        csv_files = glob.glob(os.path.join(path, "*.csv"))
        if csv_files:
            return pd.read_csv(csv_files[0])

    return None


# 주요 데이터셋 로드
datasets = {}

# 1. KOSPI200 선물 가격 데이터 (가장 최신 만기)
kospi_futures = load_dataset("domestic_futures", "kospi200_futures_daily", "101W06")
if kospi_futures is not None:
    datasets["KOSPI200_선물_가격"] = kospi_futures

# 2. KOSPI200 연속선물 데이터
kospi_continuous = load_dataset(
    "domestic_futures", "kospi200_futures_daily_continuous", "101000"
)
if kospi_continuous is not None:
    datasets["KOSPI200_연속선물"] = kospi_continuous

# 3. VKOSPI 선물 데이터
vkospi_futures = load_dataset("domestic_futures", "vkospi_futures_daily", "104W06")
if vkospi_futures is not None:
    datasets["VKOSPI_선물"] = vkospi_futures

# 4. 투자자 매매동향 데이터들
kospi_investor = load_dataset("market_data", "kospi_investor_trends_realtime", "kospi")
if kospi_investor is not None:
    datasets["KOSPI_투자자동향"] = kospi_investor

kosdaq_investor = load_dataset(
    "market_data", "kosdaq_investor_trends_realtime", "kosdaq"
)
if kosdaq_investor is not None:
    datasets["KOSDAQ_투자자동향"] = kosdaq_investor

call_investor = load_dataset(
    "market_data", "kospi200_call_investor_trends_realtime", "options"
)
if call_investor is not None:
    datasets["콜옵션_투자자동향"] = call_investor

put_investor = load_dataset(
    "market_data", "kospi200_put_investor_trends_realtime", "putoptions"
)
if put_investor is not None:
    datasets["풋옵션_투자자동향"] = put_investor

print(f"✅ 로드된 데이터셋: {len(datasets)}개")
for name, df in datasets.items():
    print(f"  - {name}: {len(df)}행 × {len(df.columns)}열")